# Test of BatchNorm Layers 

In [3]:
### used to set the interpreter searching path
import sys
target_path = '/home/jovyan/ulmo/'
sys.path.append(target_path)

In [5]:
%matplotlib inline

from importlib import reload
import os
import numpy as np
from pkg_resources import resource_filename

import pandas

import h5py
import seaborn as sns
import pandas as pd
from ulmo.ssl.train_util import modis_loader, set_model
from ulmo.ssl.train_util import Params, option_preprocess
from ulmo.ssl import latents_extraction

import torch

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

from matplotlib import pyplot as plt
from tqdm import trange

In [15]:
# Load options
opt_path = "/home/jovyan/ulmo/ulmo/runs/SSL/llc_ssl/experiments/llc_modis_2012"
opt_file = os.path.join(opt_path, 'opts.json')

In [16]:
opt = Params(opt_file)
opt = option_preprocess(opt)

In [22]:
model_dir = os.listdir(opt_path)[1]
model_name = "last.pth"
model_path = os.path.join(opt_path, model_dir, model_name)

In [67]:
using_gpu = torch.cuda.is_available()
model, _ = set_model(opt, cuda_use=using_gpu)
if not using_gpu:
    model_dict = torch.load(model_path, map_location=torch.device('cpu'))
else:
    model_dict = torch.load(model_path)

In [68]:
remove_module = True
if remove_module:
    new_dict = {}
    for key in model_dict['model'].keys():
        new_dict[key.replace('module.','')] = model_dict['model'][key]
    model.load_state_dict(new_dict)
else:
    model.load_state_dict(model_dict['model'])
print("Model loaded")

Model loaded


In [69]:
np.random.seed(0)
test_batch = np.random.randn(1, 3, 64, 64)

## Batch with size 1

In [84]:
model.eval()
with torch.no_grad():
    test_latents = model(torch.tensor(test_batch).float())

In [85]:
test_latents.shape

torch.Size([1, 256])

## Batch with size 32

In [74]:
test_batch_32 = np.repeat(test_batch, 32, axis=0)

In [81]:
model.eval()
with torch.no_grad():
    test_latents_32 = model(torch.tensor(test_batch_32).float())

## Batch with Size 64

In [99]:
test_batch_64 = np.repeat(test_batch, 64, axis=0)

In [100]:
model.eval()
with torch.no_grad():
    test_latents_64 = model(torch.tensor(test_batch_64).float())

In [103]:
test_latents[0][10: 20]

tensor([-0.0574, -0.0573,  0.0955,  0.0312, -0.0273,  0.0373,  0.0004,  0.0392,
        -0.0550,  0.1091])

In [104]:
test_latents_64[0][10: 20]

tensor([-0.0574, -0.0573,  0.0955,  0.0312, -0.0273,  0.0373,  0.0004,  0.0392,
        -0.0550,  0.1091])

---------------------------------------

In [106]:
test_batch_other = np.random.randn(63, 3, 64, 64)

In [108]:
test_batch_64_concat = np.concatenate((test_batch, test_batch_other), axis=0)

In [110]:
model.eval()
with torch.no_grad():
    test_latents_64_concat = model(torch.tensor(test_batch_64_concat).float())

In [112]:
test_latents_64_concat[0][10:20]

tensor([-0.0574, -0.0573,  0.0955,  0.0312, -0.0273,  0.0373,  0.0004,  0.0392,
        -0.0550,  0.1091])

In [113]:
test_batch_64 = np.random.randn(64, 3, 64, 64)

In [115]:
test_batch_64[24] = test_batch[0]

In [117]:
model.eval()
with torch.no_grad():
    test_latents_64 = model(torch.tensor(test_batch_64).float())

In [119]:
test_latents_64[24][10:20]

tensor([-0.0574, -0.0573,  0.0955,  0.0312, -0.0273,  0.0373,  0.0004,  0.0392,
        -0.0550,  0.1091])